### Trial for chatbot using Tavily and LangGraph and Gemini

In [ ]:
import os
import requests
from dotenv import load_dotenv
from google import genai
from google.genai import types
from groq import Groq
from linkup import LinkupClient
from openai import OpenAI
from pydantic import BaseModel
from typing import Dict

In [ ]:
TavilyKey = os.getenv("TAVILY_API_KEY")
GroqKey = os.getenv("GROQ_API_KEY")
Groq_client = Groq(api_key=GroqKey)

In [ ]:
def llm_response(client: Groq | OpenAI , sys_msg: str, user_qn: str,
              model: str = "llama-3.3-70b-versatile",
              temperature: int = 0, top_p: int = 1, max_tokens: int = 1024, is_json: bool=True) -> str:
    
    if is_json:
      json_output={
           "type": "json_schema",
           "json_schema":{
                "schema": {
                    "type": "object",
                    "properties": {
                        "Explanation": {
                            "type": "string",
                            "description": "logical analysis of question with reasoning"
                        },
                        "Answer": {
                            "type": "string",
                            "description": "Only either True or False, determined based on analysis in explanation. No other text allowed",
                            "enum": ["True", "False"]
                        }
                    },
                    "required": ["Explanation", "Answer"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    else:
      json_output=None

    chat_completion = client.chat.completions.create(
        messages=[
            # Sets system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
             "role": "system",
             "content": sys_msg
            },
            # Set a user message for the assistant to respond to.
            {
             "role": "user",
             "content": f"<question_list> {user_qn} </question_list>",
            }
        ],

        # The language model which will generate the completion.
        model=model,

        # Controls randomness: lowering results in less random completions.
        # As the temperature approaches zero, the model will become deterministic
        # and repetitive.
        temperature=temperature,

        # The maximum number of tokens to generate. Requests can use up to
        # 32,768 tokens shared between prompt and completion.
        max_completion_tokens=max_tokens,

        # Controls diversity via nucleus sampling: 0.5 means half of all
        # likelihood-weighted options are considered.
        top_p=top_p,

        # A stop sequence is a predefined or user-specified text string that
        # signals an AI to stop generating content, ensuring its responses
        # remain focused and concise. Examples include punctuation marks and
        # markers like "[end]".
        stop=None,

        # If set, partial message deltas will be sent.
        stream=False,
        response_format = json_output
        #response_format={"type":"json_object"}
    )

    return chat_completion.choices[0].message.content

In [3]:
def linkupbalance(api:str)->Dict:
    url = "https://api.linkup.so/v1/credits/balance"
    headers = {"Authorization": f"Bearer {api}"}
    response = requests.get(url, headers=headers)
    return response.json().get('balance','Nil')

In [4]:
linkupbalance(LinkupKey)

4.825

In [19]:
# Comparing Tavily vs Linkup
#Edward Lifesciences
query = "Key customers of the company Edward Lifesciences in Singapore?"

client = LinkupClient(api_key=LinkupKey)
response = client.search(
    query=query,
    depth="standard",
    output_type="sourcedAnswer",
    include_images=False)

print(response.answer)

No specific information about the key customers of Edwards Lifesciences in Singapore is available in the provided data.


In [20]:
response.sources[:3]

[LinkupSource(name='Singapore | Edwards Lifesciences', url='https://www.edwards.com/careers/locations/singapore', snippet='Situated in the east of Singapore, our facilities are well-known for their modern equipment and processes to manufacture outstanding products. In recent years, we have doubled the size of our facilities by adding extra cleanrooms and supporting different process and product transfers.\nundefined'),
 LinkupSource(name='Edwards Lifesciences | LinkedIn', url='https://sg.linkedin.com/company/edwards-lifesciences', snippet='Edwards Lifesciences | 295,827 followers on LinkedIn. Global leader in the medical device industry with a patient-focused culture of innovation | Edwards Lifesciences (NYSE: EW), is the global leader in patient-focused medical innovations for structural heart disease, as well as critical care ...\nEdwards Lifesciences | 295,827 followers on LinkedIn. Global leader in the medical device industry with a patient-focused culture of innovation | Edwards L

In [21]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TavilyKey)
response = tavily_client.search(query=query, max_results=5, topic='general', include_answer='advanced',
                               include_raw_content=True, search_depth='advanced')
response

{'query': 'Key customers of the company Edward Lifesciences in Singapore?',
 'follow_up_questions': None,
 'answer': "The retrieved data does not contain specific information about Edwards Lifesciences' key customers in Singapore. Edwards Lifesciences operates a manufacturing facility in Singapore that has been producing heart valves since 2005, primarily serving the broader Asia-Pacific region including Japan as part of their global supply chain strategy. The Singapore facility focuses on manufacturing artificial heart valves and medical devices that are distributed to healthcare providers and hospitals throughout the Asia-Pacific region, but the specific names of key customer hospitals, healthcare systems, or distributors in Singapore are not publicly disclosed in available sources. The company's customers would typically include major hospitals, cardiac surgery centers, and healthcare institutions that perform heart valve procedures and require hemodynamic monitoring equipment, but 

In [22]:
from pprint import pprint
pprint(response.get('answer'))

('The retrieved data does not contain specific information about Edwards '
 "Lifesciences' key customers in Singapore. Edwards Lifesciences operates a "
 'manufacturing facility in Singapore that has been producing heart valves '
 'since 2005, primarily serving the broader Asia-Pacific region including '
 'Japan as part of their global supply chain strategy. The Singapore facility '
 'focuses on manufacturing artificial heart valves and medical devices that '
 'are distributed to healthcare providers and hospitals throughout the '
 'Asia-Pacific region, but the specific names of key customer hospitals, '
 'healthcare systems, or distributors in Singapore are not publicly disclosed '
 "in available sources. The company's customers would typically include major "
 'hospitals, cardiac surgery centers, and healthcare institutions that perform '
 'heart valve procedures and require hemodynamic monitoring equipment, but '
 'identifying the specific key customers would require access to Edwar